In [1]:
import torch
from torch import nn 
import numpy as np
from scipy.special import softmax
import random
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sampling_method import search
from cross_entropy_sampling import cross_entropy_time
from gradient_descent_diagram import model_arrival, train
from plotly.subplots import make_subplots

/Users/bb/opt/anaconda3/envs/ev1/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
np.random.seed(0)
gau = np.clip(np.random.normal(12, 5, 100), 0, 48)
gau = [len([g for g in gau if i <= g < i+1])/100 for i in range(48)]

In [ ]:
solver = cross_entropy_time(48,  balanced_factor=10,  Nsamples=200, rho=0.05, maxite=100, total_vehicles=100, P=10, decay_rate=0.06, B=6, maxu=0.5, method='fix', seed=0)
solver.x_init = init_soc
solver.search(show_animation=False)

In [ ]:
_ = solver.get_function_value(solver.mu, verbose=True)

In [11]:
solver_grad1 = model_arrival(strategy='fix',balanced_factor=20, e=48, u=0.5, decay_rate=0.0, total_vehicles=100,  battery_capacity=6, power_capacity=10, T=48, seed=0)
solver_grad1.y.data = torch.from_numpy(np.array([0.]*48))
solver_grad1 = train(solver=solver_grad1, balanced_factor=5)
y1 = solver_grad1.y.detach().numpy()

  0%|          | 1/500 [00:01<12:14,  1.47s/it]

222.2645649067796 1737.4999999999995


100%|██████████| 500/500 [13:48<00:00,  1.66s/it]


In [13]:
loss = solver_grad1.loss
fig = make_subplots(rows=1, cols=2)
fig.add_trace(go.Scatter(y=loss,name='loss'),row=1,col=1)
fig.add_trace(go.Scatter(y=softmax(y1)*100,name='optimal arrival'),row=1,col=2)
fig.update(layout={
    'template':'simple_white',
    'legend':{'yanchor':'bottom','y':1.2,'xanchor':'left','x':0.0,'orientation':'h'},
     'width':1000,
    'height':500,
    "font": {"size": 20},
})
fig.show()

In [14]:
fig.write_image("image/random_soc2/ES_P20.pdf")

In [ ]:
solver_grad1 = model_arrival(strategy='fix',balanced_factor=10, e=48, u=0.5, decay_rate=0.06, total_vehicles=100,  battery_capacity=6, power_capacity=10, T=48, seed=0)
init_soc = solver_grad1.x.detach().numpy()
solver_grad1.y.data = torch.from_numpy(np.array([0.]*48))
solver_grad1 = train(solver=solver_grad1, balanced_factor=20)
y2 = solver_grad1.y.detach().numpy()

In [ ]:
_ = solver.get_function_value(y1, verbose=True)

In [ ]:
_ = solver.get_function_value([0]*48, verbose=True)
_ = solver.get_function_value(np.array(gau), verbose=True, process=True)
solver_grad_ = solver.get_function_value(y1, verbose=True)
solver_ = solver.get_function_value(solver.mu, verbose=True)
# a = [0.11]+[0.01]*7
# _ = solver.get_function_value(np.array(a*5+[0.1]+[0]*7), verbose=True, process=True)

In [ ]:
1912.49/100*15, 3424.0/100*15, 1132.8/100*15, 1007.93/100*15

In [ ]:
_ = solver.get_function_value(np.array([0.1]+[0]*7+[0.1]+[0]*7+[0.1]+[0]*7+[0.1]+[0]*7+[0.1]+[0]*7+[0.1]+[0]*6+[0.4]), verbose=True, process=True)

In [ ]:
600/100*15

In [ ]:
n = np.array(hand) * 100
method = 'fix'
T = 48
x_init, final_energy = 0.2*6*n, 0.9*6
x_step = np.copy(x_init)
u_record = [0 for _ in range(T)]
stay_record = [0 for _ in range(T)]
stay, charging_cost, P, maxu = 0, 0, 5, 0.5
for k in range(T):
    u = np.zeros(T)
    V = [i for i in range(k+1) if x_step[i] <= final_energy*n[i] - 1e-3]
    print(V)
    stay_record[k] = sum([n[i] for i in V])
    totalev = np.sum([n[i] for i in V])
    stay += totalev
    if method == 'fix': budget = P
    for i in V:
        if method == 'ES':
            u[i] = min(min(P/totalev*n[i], n[i]*(maxu-decay_rate*x_step[i]/n[i])), final_energy*n[i] - x_step[i])
        elif method == 'fix':
            u[i] = min(n[i]*maxu, final_energy*n[i] - x_step[i])
            if budget < u[i]: 
                u[i] = budget
                break
            budget -= u[i]
    x_step = x_step + u
    charging_cost = charging_cost + np.sum(u)*TOU_price[k]
    u_record[k] = sum(u)

In [ ]:
TOU_price = np.zeros(48)
TOU_price[:12]=0.51
TOU_price[12:]=0.43
fig = make_subplots(rows=1, cols=2, subplot_titles=("Gradient","Sampling"))

#solver_grad_ = solver1.get_function_value(y, verbose=True)
solver_ = solver.get_function_value(solver.mu, verbose=True)
#fig.add_trace(go.Scatter(y=u_g, showlegend=False, marker_color='#8cc1e6',yaxis='y3', line={'width':3}),row=2, col=1)
fig.add_trace(go.Scatter(y=softmax(y1)*100, name='Arrival', marker_color='#ffa05a',line={'width':3}),row=1, col=1)
#fig.add_trace(go.Scatter(y=solver1.stay_record, name='Available vehicles', marker_color='#8cc1e6',line={'width':3}),row=1, col=1)
fig.add_trace(go.Scatter(y=softmax(solver.mu)*100, showlegend=False, marker_color='#ffa05a',line={'width':3}),row=1, col=2)
fig.add_trace(go.Scatter(y=solver.stay_record, showlegend=False, marker_color='#8cc1e6',line={'width':3}),row=1, col=2)


fig.update(layout={
    'xaxis':{'title':'Time (Hours)', 'showline':True, 'tickmode': 'array', 'tickvals': [0, 16, 32], 'ticktext': ['18:00','22:00','2:00']},
    'xaxis2':{'showline':True, 'tickmode': 'array', 'tickvals': [0, 16, 32], 'ticktext': ['18:00','22:00','2:00']},
    'xaxis3':{'showline':True, 'tickmode': 'array', 'tickvals': [0, 16, 32], 'ticktext': ['18:00','22:00','2:00']},
    'xaxis4':{'showline':True, 'tickmode': 'array', 'tickvals': [0, 16, 32], 'ticktext': ['18:00','22:00','2:00']},
    'xaxis5':{'showline':True, 'tickmode': 'array', 'tickvals': [0, 16, 32], 'ticktext': ['18:00','22:00','2:00']},
    'xaxis6':{'showline':True, 'tickmode': 'array', 'tickvals': [0, 16, 32], 'ticktext': ['18:00','22:00','2:00']},
    'xaxis7':{'showline':True, 'tickmode': 'array', 'tickvals': [0, 16, 32], 'ticktext': ['18:00','22:00','2:00']},
    'yaxis5':{'title': 'Charging Power (kWh)', 'range':[0,15]},
    'yaxis6':{ 'range':[0,16]},
    'yaxis':{'title': 'Number of Vehicles'},
    "font": {"size": 20},
    'template':'simple_white',
    'legend':{'yanchor':'bottom','y':1.2,'xanchor':'left','x':0.0,'orientation':'h'},
     'width':1200,
    'height':560
})
fig.update_annotations(font_size=20)
fig.show()

In [ ]:
420/418-1, 1-135/174.3

In [ ]:
fig.write_image("image/compare_es_p20.pdf")

In [ ]:
fig.write_image("image/stay_comparison_sample.pdf")

In [ ]:
sum(u), sum(u_g), sum(u_u), sum(u_p)

In [ ]:
TOU_price = np.zeros(48)
TOU_price[:12]=0.51
TOU_price[12:]=0.43
fig = make_subplots(rows=1, cols=4, subplot_titles=("Gradient", "Sampling","Gaussian","Uniform"))

fig.add_trace(go.Scatter(y=softmax(y2)*100, showlegend=False, marker_color='#ffa05a',line={'width':3}),row=1, col=1)
#fig.add_trace(go.Scatter(y=softmax(y1)*100, showlegend=False, line={'width':3}),row=1, col=1)
#fig.add_trace(go.Scatter(y=softmax(y2)*100, showlegend=False, line={'width':3}),row=1, col=1)
#fig.add_trace(go.Scatter(y=softmax(y3)*100, showlegend=False, line={'width':3}),row=1, col=1)
#fig.add_trace(go.Scatter(y=u_g, showlegend=False, marker_color='#8cc1e6',yaxis='y3', line={'width':3}),row=2, col=1)
fig.add_trace(go.Scatter(y=softmax(solver.mu)*100, showlegend=False, marker_color='#ffa05a',line={'width':3}),row=1, col=2)
#fig.add_trace(go.Scatter(y=softmax(solver1.mu)*100, showlegend=False, line={'width':3}),row=1, col=2)
#fig.add_trace(go.Scatter(y=softmax(solver2.mu)*100, showlegend=False, line={'width':3}),row=1, col=2)
#fig.add_trace(go.Scatter(y=u, showlegend=False, marker_color='#8cc1e6',yaxis='y3', line={'width':3}),row=2, col=2)
fig.add_trace(go.Scatter(y=np.array(gau)*100, showlegend=False, marker_color='#ffa05a',line={'width':3}),row=1, col=3)
#fig.add_trace(go.Scatter(y=u_p, showlegend=False, marker_color='#8cc1e6', yaxis='y4', line={'width':3}),row=2, col=3)
fig.add_trace(go.Scatter(y=softmax([0]*48)*100, showlegend=False, marker_color='#ffa05a',line={'width':3}),row=1, col=4)
#fig.add_trace(go.Scatter(y=u_u, showlegend=False, marker_color='#8cc1e6', yaxis='y4', line={'width':3}),row=2, col=4)

#fig.add_trace(go.Scatter(y=softmax([0]*45)*100, showlegend=False, marker_color='#ffa05a',line={'width':3}),row=1, col=3)
#fig.add_trace(go.Scatter(y=np.array(poisson)*100, showlegend=False, marker_color='#ffa05a',line={'width':3}),row=1, col=4)


fig.update(layout={
    'xaxis':{'title':'Time (Hours)', 'showline':True, 'tickmode': 'array', 'tickvals': [0, 16, 32], 'ticktext': ['18:00','22:00','2:00']},
    'xaxis2':{'showline':True, 'tickmode': 'array', 'tickvals': [0, 16, 32], 'ticktext': ['18:00','22:00','2:00']},
    'xaxis3':{'showline':True, 'tickmode': 'array', 'tickvals': [0, 16, 32], 'ticktext': ['18:00','22:00','2:00']},
    'xaxis4':{'showline':True, 'tickmode': 'array', 'tickvals': [0, 16, 32], 'ticktext': ['18:00','22:00','2:00']},
    'xaxis5':{'showline':True, 'tickmode': 'array', 'tickvals': [0, 16, 32], 'ticktext': ['18:00','22:00','2:00']},
    'xaxis6':{'showline':True, 'tickmode': 'array', 'tickvals': [0, 16, 32], 'ticktext': ['18:00','22:00','2:00']},
    'xaxis7':{'showline':True, 'tickmode': 'array', 'tickvals': [0, 16, 32], 'ticktext': ['18:00','22:00','2:00']},
    'yaxis5':{'title': 'Charging Power (kWh)', 'range':[0,15]},
    'yaxis6':{ 'range':[0,16]},
    'yaxis':{'title': 'Number of Vehicles'},
    "font": {"size": 20},
    'template':'simple_white',
    'legend':{'yanchor':'bottom','y':1.2,'xanchor':'left','x':0.0,'orientation':'h'},
     'width':1200,
    'height':360
})
fig.update_annotations(font_size=20)
fig.show()

In [ ]:
fig.write_image("image/random_soc/stay_ES_p20_v.pdf")

In [ ]:
sum(softmax(y1)),sum(softmax(y2))

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=softmax(y1),name='P=20'))
fig.add_trace(go.Scatter(y=softmax(y2),name='P=40'))
fig.update(layout={
    'xaxis':{'title':'Time (Hours)', 'showline':True, 'tickmode': 'array', 'tickvals': [0, 16, 32], 'ticktext': ['18:00','22:00','2:00']},
    'yaxis':{'title': 'fraction'},
    "font": {"size": 20},
    'template':'simple_white',
    'legend':{'yanchor':'bottom','y':1.2,'xanchor':'left','x':0.0,'orientation':'h'},
     'width':400,
    'height':360
})
fig.show()

In [ ]:
fig.write_image("image/random_soc/FIFS_compare.pdf")

In [ ]:
color = ['red','blue','yellow','gray','orange','green']
fig = make_subplots(rows=1, cols=2, subplot_titles=("Solution","Statistics"))

fig.add_trace(go.Scatter(y=softmax(y)*100,  name='grad1', marker_color=color[0],line={'width':3}),row=1, col=1)
fig.add_trace(go.Scatter(y=softmax(y1)*100,  name='grad2', marker_color=color[1], line={'width':3}),row=1, col=1)
fig.add_trace(go.Scatter(y=softmax(y2)*100,  name='grad3', marker_color=color[2],line={'width':3}),row=1, col=1)
#fig.add_trace(go.Scatter(y=softmax(y3)*100, showlegend=False, line={'width':3}),row=1, col=1)
#fig.add_trace(go.Scatter(y=u_g, showlegend=False, marker_color='#8cc1e6',yaxis='y3', line={'width':3}),row=2, col=1)
#fig.add_trace(go.Scatter(y=softmax(solver.mu)*100, showlegend=False, marker_color='#ffa05a',line={'width':3}),row=1, col=1)
fig.add_trace(go.Scatter(y=softmax(solver.mu)*100, name='sample1',  marker_color=color[3],line={'width':3}),row=1, col=1)
fig.add_trace(go.Scatter(y=softmax(solver1.mu)*100,  name='sample2', marker_color=color[4],line={'width':3}),row=1, col=1)
fig.add_trace(go.Scatter(y=softmax(solver2.mu)*100,  name='sample3', marker_color=color[5],line={'width':3}),row=1, col=1)
staytime = [0.58,1.14,1.14,0.896,0.569,0.923]
#staytime = [s/100*15 for s in staytime]
energy = [419.97,419.98,419.99,419.71,419.42,419.40]
for i in range(6):  
    fig.add_trace(go.Scatter(y=[energy[i]],showlegend=False,x=[staytime[i]],marker_color=color[i],marker={'size':12}),row=1,col=2)

fig.update(layout={
    'xaxis':{'title':'Time (Hours)', 'showline':True, 'tickmode': 'array', 'tickvals': [0, 16, 32], 'ticktext': ['18:00','22:00','2:00']},
    'xaxis2':{'showline':True, 'title':'PFL', 'range':[0.3,1.5]},
    'yaxis':{'title': 'Number of Vehicles'},
    'yaxis2':{'title': 'Delivered Energy(kWh)','range':[0,425]},
    "font": {"size": 20},
    'template':'simple_white',
    'legend':{'yanchor':'bottom','y':1.2,'xanchor':'left','x':0.0,'orientation':'h'},
     'width':1200,
    'height':600
})
fig.update_annotations(font_size=20)
fig.show()

In [ ]:
fig.write_image("image/sensitivity_ES_pfl.pdf")